In [1]:
import pandas as pd
import numpy as np
import re

In [254]:
# Read ygs.txt into a dataframe
#price_of_products_df = pd.read_excel('data_files/ygs.xlsx')
price_of_products_df = pd.read_excel('data_files/ygs_.xlsx')

In [3]:
reserve = price_of_products_df.copy()

In [302]:
price_of_products_df = reserve.copy()

In [255]:
price_of_products_df.reset_index(inplace=True)

In [256]:
wrong_positions = price_of_products_df[price_of_products_df['Curr.'] != 'EUR']

In [257]:
wrong1 = wrong_positions[wrong_positions['Column1'] == 'EUR']

wrong2 = wrong_positions[wrong_positions['Column1'].isna()]

wrong_temp = wrong_positions[~wrong_positions['Column1'].isna()]

wrong3 = wrong_temp[wrong_temp['Column1'] != 'EUR']

In [258]:
len(wrong_positions) == len(wrong1) + len(wrong2) + len(wrong3)

True

In [ ]:
# fixing first wrong data

# Shift the values in each column by 1
wrong1['Value'] = wrong1['Description_2'] 
wrong1['Description_2'] = wrong1['Del. Time'] 
wrong1['Del. Time'] = wrong1['Min.Order qty']
wrong1['DLP'] = wrong1['Curr.']
wrong1['Curr.'] = wrong1['Column1']
wrong1['Column1'] = ""

wrong1['DLP'] = wrong1['DLP'].astype(str)
wrong1['DLP'] = wrong1['DLP'].str.strip()

In [ ]:
# Create a copy of the DataFrame slice using .copy()
wrong1_copy = wrong1.loc[:, 'Value'].copy()

# Assign values from 'Description_2' column to the copied 'Value' column
wrong1_copy.loc[:] = wrong1.loc[:, 'Description_2']

# Update the original DataFrame with the modified copy
wrong1.loc[:, 'Value'] = wrong1_copy

In [261]:
# fixing wrong data #2 

wrong2 = wrong2.dropna(axis=1)
wrong2[['Description', 'Characteristic', 'Description_1', 'Value', 'Description_2',
       'Del. Time', 'Min.Order qty', 'DLP', 'Curr.', 'Column1', '_3']] = wrong2['Description'].str.split('\t', expand=True)

wrong2['DLP'] = wrong2['DLP'].astype(str)
wrong2['DLP'] = wrong2['DLP'].str.strip()

In [ ]:
# fixing the third case of wrong data

# Split the column value into two columns
wrong3['Description_2'] = wrong3['Curr.']

split_values = wrong3['Curr.'].str.split(' ', 1, expand=True)
wrong3['Value'] = split_values.iloc[:, 0]

wrong3['DLP'] = wrong3['_3']
wrong3['Min.Order qty'] = wrong3['Column1']
wrong3['Curr.'] = 'EUR'

wrong3['DLP'] = wrong3['DLP'].astype(str)
wrong3['DLP'] = wrong3['DLP'].str.strip()

In [263]:
right_positions = price_of_products_df[~(price_of_products_df['Curr.'] != 'EUR')]

In [264]:
final = pd.concat([right_positions, wrong1, wrong2, wrong3])
final_int = final.copy()

In [265]:
final = final.sort_values(by='index')
final = final.loc[:, ['Material/Model', 'Product Hierarchy', 'Description',
       'Characteristic', 'Description_1', 'Value', 'Description_2',
       'Del. Time', 'Min.Order qty', 'DLP', 'Curr.']]

final = final.rename(columns={'Material/Model': 'Model', 
                        'Description': 'Model_Description',                          
                        'Description_2': 'Option_Description', 
                        'Value': 'Option',
                        'DLP': 'Price',
                        'Product Hierarchy': 'Group'})

# extract the first 5 characters of the 'BU' column
final['Group'] = final['Group'].str.slice(stop=5)


# keep only the desired columns
final = final[['Model', 'Group', 'Model_Description', 'Characteristic', 'Option', 'Option_Description', 'Price']]

final['Characteristic'] = final['Characteristic'].astype('str')
# update values
final['Characteristic'] = final['Characteristic'].apply(lambda x: x.split('_', 1)[1] if '_' in x else x)

final['Option'] = final['Option'].astype('str')
final['Option_Description'] = final['Option_Description'].astype('str')

# Replace 'nan' values with None in a specific column
final['Characteristic'] = final['Characteristic'].replace('nan', np.nan)
final['Option'] = final['Option'].replace('nan', np.nan)
final['Option_Description'] = final['Option_Description'].replace('nan', np.nan)

final = final.reset_index()

In [ ]:
# Convert string values to float in a specific column and create a mask for errors
mask = pd.to_numeric(final['Price'], errors='coerce').isna()

# Filter the DataFrame to include only the rows with errors
error_df = final[mask]
normal_df = final[~mask]

normal_df['Price'] = pd.to_numeric(normal_df['Price'], errors='coerce')

error_df['Price'] = error_df['Price'].str.replace('.','').str.replace('-', '').str.replace(',', '.').astype(float)

In [16]:
final = pd.concat([normal_df, error_df])
final = final.sort_values(by='index')
final = final[['Model', 'Group', 'Model_Description', 'Characteristic', 'Option', 'Option_Description', 'Price']]

In [318]:
writer = pd.ExcelWriter('data_files/final.xlsx')
final.to_excel(writer, sheet_name='final')
writer.close()

In [348]:
class ProductModel:
    def __init__(self, model_code, model_group, model_description, model_price, characteristics=None, options=None):
        self.model_code = model_code
        self.model_group = model_group
        self.model_description = model_description
        self.model_price = model_price
        self.characteristics = characteristics or []
        self.options = options or []

    def add_characteristic(self, name, description, price):
        self.characteristics.append({
            'name': name,
            'description': description,
            'price': price
        })

    def add_option(self, name, description, price):
        self.options.append({
            'name': name,
            'description': description,
            'price': price
        })
        
    def define_cost(self, ms_code):
        mis_val = []
        group_dict = extract_groups(ms_code)
        self.total_cost = 0
        keys = list(group_dict.keys())
        values = list(group_dict.values())
        
        if group_dict[keys[0]] != self.model_code:
            return 0
        else:
            self.total_cost += self.price
            values.remove(self.model_code)
            counter = 0
            for char in self.characteristics:
                if str(char['name']) in values and str(char['name']) == values[0]:
                    self.total_cost += char['price']
                    #print(str(char['name']), char['price'])
                    values.remove(str(char['name']))
                    #print(values)
            for opt in self.options:
                if str(opt['name']) in values and str(opt['name']) == values[0]:
                    self.total_cost += opt['price']
                    #print(str(opt['name']), opt['price'])
                    values.remove(str(opt['name']))
            missing_values = values
            if missing_values:
                mis_val = mis_val + missing_values
        
        return round(self.total_cost,2), mis_val
        
    def __str__(self):
        return f'Model code: {self.model_code}, Description: {self.model_description}, Price: {self.model_price}\n Characteristics: {self.characteristics}\n Options: {self.options}'

In [169]:
class Model:
    def __init__(self, model_code, model_group, model_description, model_price):
        self.model_code = model_code
        self.model_group = model_group
        self.model_description = model_description
        self.model_price = model_price
        self.characteristics = []
        self.option_characteristics = []

    def add_characteristic(self, characteristic):
        self.characteristics.append(characteristic)
        
    def add_option_characteristic(self, characteristic):
        self.option_characteristics.append(characteristic)
        
    def __str__(self):
        return f'Model name: {self.model_code}, Price: {self.model_price}'
        
    
    def calculate_price(self, code):
        groups_dict = extract_groups(code)
        total_price = self.model_price  # Start with the model's base price

        # Check if the model_code matches, add the model's price
        if self.model_code == groups_dict.get('Model'):
            
            exclusion_characteristics = []
            not_in_base = []
            # Check groups in order
            for key, value in list(groups_dict.items())[1:]:
                print(key, value)
                found = False
                
                for characteristic in list(self.characteristics):
                    for option in characteristic.options:
                        if option.option_name == value and found == False and characteristic.group not in exclusion_characteristics:
                            total_price += option.option_price
                            print(characteristic, option)
                            found = True
                            exclusion_characteristics.append(characteristic.group)
                            break
                            
                for characteristic in list(self.option_characteristics):
                    for option in characteristic.options:
                        if option.option_name == value and found == False and characteristic.group not in exclusion_characteristics:
                            total_price += option.option_price
                            print(characteristic, option)
                            found = True
                            exclusion_characteristics.append(characteristic.group)
                            break
                
                if found == False:
                    not_in_base.append(value)
            print(not_in_base)
                            
        total_price = round(total_price, 2)
                            
        return total_price
                


class Characteristic:
    def __init__(self, group, options=None):
        self.group = group
        self.options = options if options is not None else []

    def add_option(self, option):
        self.options.append(option)
        
    def __str__(self):
        return f'Characteristic group: {self.group}'


class Option:
    def __init__(self, option_name, option_description, option_price):
        self.option_name = option_name
        self.option_description = option_description
        self.option_price = option_price
    def __str__(self):
        return f'Option name: {self.option_name}, Price: {self.option_price}'

In [ ]:
import pandas as pd

# Assuming you already have a DataFrame called df

df = final[final['Model'] == 'EJX110A']
df['Characteristic'] = df['Characteristic'].astype(str)

# Create a dictionary to store the Model objects
models_dict = {}

for _, row in df.iterrows():
    model_code = row['Model']
    model_group = row['Group']
    model_description = row['Model_Description']
    model_price = row['Price']

    # Check if the model already exists in the dictionary, if not, create a new Model object
    if model_code not in models_dict:
        model = Model(model_code, model_group, model_description, model_price)
        models_dict[model_code] = model
    else:
        model = models_dict[model_code]
    
    # Determine the length of the characteristic group
    characteristic_group = row['Characteristic']
        

    # If length is 2, it represents characteristics and options
    if pd.isna(characteristic_group) or characteristic_group == 'nan':
        pass
    elif len(characteristic_group) == 2:
        characteristic = Characteristic(row['Characteristic'])
        option = Option(row['Option'], row['Option_Description'], row['Price'])
        characteristic.add_option(option)
        model.add_characteristic(characteristic)
    
    # If length is 3 or longer, it represents an option with sub-options
    elif len(characteristic_group)  >= 3:
        characteristic = Characteristic(row['Characteristic'])
        option = Option(row['Option'], row['Option_Description'], row['Price'])
        characteristic.add_option(option)
        model.add_option_characteristic(characteristic)

In [ ]:
models_dict['model'].calculate_price('ms_code')

In [ ]:
# Access the Model objects from the dictionary
for model_code, model in models_dict.items():
    print('Model Code:', model.model_code)
    print('Model Group:', model.model_group)
    print('Model Description:', model.model_description)
    print('Model Price:', model.model_price)

    # Access the characteristics for each model
    for characteristic in model.characteristics:
        print('Characteristic Group:', characteristic.group)

        # Access the options for each characteristic
        for option in characteristic.options:
            print('Option Name:', option.option_name)
            print('Option Description:', option.option_description)
            print('Option Price:', option.option_price)

        print('---')
        
    # Access the characteristics for each model
    for characteristic in model.option_characteristics:
        print('Characteristic Group:', characteristic.group)

        # Access the options for each characteristic
        for option in characteristic.options:
            print('Option Name:', option.option_name)
            print('Option Description:', option.option_description)
            print('Option Price:', option.option_price)


In [222]:
# Define the regex pattern to match separators ('-' and '/')
pattern = r'([-/][^-/]+|^[^-/]+)'


def extract_groups(code):
    code = str(code)
    # Split the code into groups using regex
    groups = re.findall(pattern, code)
    # Split groups containing the first dash on a letter
    
    i = 1
    while i < len(groups):
        if '-' in groups[i] and len(groups[i]) > 3:
            group_parts = [char for char in groups[i]]
            # insert a list of elements at a specific position
            groups[i:i+1] = group_parts
            i += len(group_parts) - 1
        else:
            i += 1
            
    
    for i in range(len(groups)-2):
        if groups[i] == '-':
            groups[i:i+2] = [''.join(groups[i:i+2])]
    
    # Combine all the groups into a dictionary

    groups_dict = {'Model': groups[0]}
    groups_dict.update({'group{}'.format(i): groups[i] for i in range(1, len(groups))})
    return groups_dict

class Model:
    def __init__(self, model_code, model_group, model_description, model_price):
        self.model_code = model_code
        self.model_group = model_group
        self.model_description = model_description
        self.model_price = model_price
        self.characteristics = []
        self.option_characteristics = []

    def add_characteristic(self, characteristic):
        self.characteristics.append(characteristic)
        
    def add_option_characteristic(self, characteristic):
        self.option_characteristics.append(characteristic)
        
    def __str__(self):
        return f'Model name: {self.model_code}, Price: {self.model_price}'
        
    
    def calculate_price(self, code):
        groups_dict = extract_groups(code)
        total_price = self.model_price  # Start with the model's base price

        # Check if the model_code matches, add the model's price
        if self.model_code == groups_dict.get('Model'):
            
            exclusion_characteristics = []
            not_in_base = []
            # Check groups in order
            for key, value in list(groups_dict.items())[1:]:
                found = False
                
                for characteristic in list(self.characteristics):
                    for option in characteristic.options:
                        if option.option_name == value and found == False and characteristic.group not in exclusion_characteristics:
                            total_price += option.option_price
                            found = True
                            exclusion_characteristics.append(characteristic.group)
                            break
                            
                for characteristic in list(self.option_characteristics):
                    for option in characteristic.options:
                        if option.option_name == value and found == False and characteristic.group not in exclusion_characteristics:
                            total_price += option.option_price
                            found = True
                            exclusion_characteristics.append(characteristic.group)
                            break
                
                if found == False:
                    not_in_base.append(value)
                            
        total_price = round(total_price, 2)
                            
        return total_price, not_in_base
                


class Characteristic:
    def __init__(self, group, options=None):
        self.group = group
        self.options = options if options is not None else []

    def add_option(self, option):
        self.options.append(option)
        
    def __str__(self):
        return f'Characteristic group: {self.group}'


class Option:
    def __init__(self, option_name, option_description, option_price):
        self.option_name = option_name
        self.option_description = option_description
        self.option_price = option_price
    def __str__(self):
        return f'Option name: {self.option_name}, Price: {self.option_price}'

In [223]:
def create_model_dict_from_df(input_df):
    df = input_df.copy()
    df['Characteristic'] = df['Characteristic'].astype(str)

    # Create a dictionary to store the Model objects
    models_dict = {}

    for _, row in df.iterrows():
        model_code = row['Model']
        print(model_code)
        model_group = row['Group']
        model_description = row['Model_Description']
        model_price = row['Price']

        # Check if the model already exists in the dictionary, if not, create a new Model object
        if model_code not in models_dict:
            model = Model(model_code, model_group, model_description, model_price)
            models_dict[model_code] = model
        else:
            model = models_dict[model_code]

        # Determine the length of the characteristic group
        characteristic_group = row['Characteristic']


        # If length is 2, it represents characteristics and options
        if pd.isna(characteristic_group) or characteristic_group == 'nan':
            pass
        elif len(characteristic_group) == 2:
            characteristic = Characteristic(row['Characteristic'])
            option = Option(row['Option'], row['Option_Description'], row['Price'])
            characteristic.add_option(option)
            model.add_characteristic(characteristic)

        # If length is 3 or longer, it represents an option with sub-options
        elif len(characteristic_group)  >= 3:
            characteristic = Characteristic(row['Characteristic'])
            option = Option(row['Option'], row['Option_Description'], row['Price'])
            characteristic.add_option(option)
            model.add_option_characteristic(characteristic)
            
    return models_dict

In [ ]:
models = create_model_dict_from_df(final)

In [193]:
# Read ygs.txt into a dataframe
ms_codes = pd.read_excel('data_files/ms_codes.xlsx')

In [242]:
ms_codes['price'] = 0
ms_codes['check'] = ''


for i, row in ms_codes.iterrows():
    
    material = str(row['material'])
    ms_code = str(row['ms_code'])

    
    try:
        price, not_in = models[material].calculate_price(ms_code)
        if not_in == []:
            not_in = 'ok'
            
        ms_codes.loc[i, 'price'] = price
        ms_codes.loc[i, 'check'] = not_in
        
    except:
        
        price = 0
        not_in = ''
        ms_codes.loc[i, 'price'] = price
        ms_codes.loc[i, 'check'] = not_in

In [266]:
writer = pd.ExcelWriter('data_files/final_int.xlsx')
final_int.to_excel(writer, sheet_name='final')
writer.close()